In [1]:
import pandas as pd

test_df: pd.DataFrame = pd.read_pickle('data/test.pkl')

test_df = test_df.groupby('id').apply(lambda x: pd.Series({
    'history': x['history'].iloc[0],
    'positives': set(x['ID'])
}), include_groups=False)
test_df

,history,positives
id,,
1005_2024-11-06,"[[26, 0], [5, 1], [16, 1]]",{6}
1008_2024-11-04,"[[2, 1], [28, 0], [18, 1]]",{6}
1008_2024-11-05,"[[2, 1], [28, 0], [18, 1], [28, 0], [6, 1], [1...","{10, 30, 14}"
1008_2024-11-06,"[[2, 1], [28, 0], [18, 1], [28, 0], [6, 1], [1...","{0, 18, 11}"
1008_2024-11-07,"[[28, 0], [6, 1], [19, 0], [30, 1], [14, 1], [...","{5, 15}"
...,...,...
957_2024-11-06,"[[1, 0], [9, 0], [19, 0], [22, 0], [0, 0], [7,...",{2}
957_2024-11-07,"[[22, 0], [0, 0], [7, 0], [21, 0], [15, 0], [2...",{2}
972_2024-11-01,"[[21, 0], [17, 0], [0, 1], [12, 0], [26, 0], [...",{1}


In [ ]:
import torch
import numpy as np
from 

def rank(x):
    history = torch.from_numpy(x['history']).view(1, -1, 2).to(DEVICE)
    with torch.no_grad():
        scores = model(history).view(-1).detach().cpu().numpy()
    return np.argsort(scores)[::-1]

def random_rank(x):
    return np.random.permutation(len(missions))

test_df['gru4rec'] = test_df.progress_apply(rank, axis=1)
test_df['random'] = test_df.progress_apply(random_rank, axis=1)
test_df

In [ ]:
def recall(x, model, k=5):
    top_k = set(x[model][:k])
    return len(x['positives'] & top_k) / len(x['positives'])

def hit_rate(x, model, k=5):
    top_k = set(x[model][:k])
    return len(x['positives'] & top_k) > 0

pd.DataFrame({
    'gru4rec': {
        'recall@5': test_df.progress_apply(lambda x: recall(x, 'gru4rec'), axis=1).mean(),
        'recall@10': test_df.progress_apply(lambda x: recall(x, 'gru4rec', k=10), axis=1).mean(),
        'hit_rate@5': test_df.progress_apply(lambda x: hit_rate(x, 'gru4rec'), axis=1).mean(),
        'hit_rate@10': test_df.progress_apply(lambda x: hit_rate(x, 'gru4rec', k=10), axis=1).mean()
    },
    'random': {
        'recall@5': test_df.progress_apply(lambda x: recall(x, 'random'), axis=1).mean(),
        'recall@10': test_df.progress_apply(lambda x: recall(x, 'random', k=10), axis=1).mean(),
        'hit_rate@5': test_df.progress_apply(lambda x: hit_rate(x, 'random'), axis=1).mean(),
        'hit_rate@10': test_df.progress_apply(lambda x: hit_rate(x, 'random', k=10), axis=1).mean()
    }
})